In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import sys
sys.path.insert(1, 'C:/Users/pnknv/Documents/GitHub/yadisk_loader')
from main_utils import *

In [13]:
import json

In [3]:
ssh_host, ssh_port, ssh_username, ssh_password, \
database_username, database_password, database_name, \
localhost, localhost_port, table_name, ya_token, ya_api, ya_link = get_config()

In [4]:
sql_server, sql_engine = get_sql_engine()

In [6]:
def get_dadata(engine):
    dadata_query = f'SELECT id, data FROM dadata_houses'
    try:
        con_obj = engine.connect()
        dadata_db = pd.read_sql(text(dadata_query), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(traceback.format_exc())
        dadata_db = None
        exc_code = exc.code
    return dadata_db, exc_code

In [7]:
dadata = get_dadata(sql_engine)

In [10]:
dadata = dadata[0]

In [47]:
def get_json(json_str):
    try:
        return json.loads(json_str)
    except:
        try:
            return json.loads(json_str.replace("'", '"').replace('None', 'null'))
        except:
            try:
                return json.loads(json_str.replace('\\xa0', '').replace("'", '"').replace('None', 'null'))
            except:
                return None

In [48]:
dadata['json_data'] = dadata.data.apply(lambda x: get_json(x))

In [49]:
dadata[dadata.json_data.isna()]

,id,data,json_data


In [52]:
dadata['addr'] = dadata.json_data.apply(lambda x: x['source'])

In [53]:
dadata

,id,data,json_data,addr
0,1,"{""source"":""Челябинская область; Челябинск; р-н...",{'source': 'Челябинская область; Челябинск; р-...,Челябинская область; Челябинск; р-н Трактороза...
1,2,"{""source"":""Челябинская область; Челябинск; р-н...",{'source': 'Челябинская область; Челябинск; р-...,Челябинская область; Челябинск; р-н Трактороза...
2,3,"{""source"":""Челябинская область; Челябинск; р-н...",{'source': 'Челябинская область; Челябинск; р-...,Челябинская область; Челябинск; р-н Трактороза...
3,4,"{""source"":""Челябинская область; Челябинск; р-н...",{'source': 'Челябинская область; Челябинск; р-...,Челябинская область; Челябинск; р-н Трактороза...
4,5,"{""source"":""Челябинская область; Челябинск; р-н...",{'source': 'Челябинская область; Челябинск; р-...,Челябинская область; Челябинск; р-н Трактороза...
...,...,...,...,...
325006,379195,{'source': 'Новосибирск; ул. Большевистская; с...,{'source': 'Новосибирск; ул. Большевистская; с...,Новосибирск; ул. Большевистская; стр. 14
325007,379196,{'source': 'Новосибирск; ул. Большевистская; с...,{'source': 'Новосибирск; ул. Большевистская; с...,Новосибирск; ул. Большевистская; стр. 14
325008,379197,{'source': 'Новосибирск; пр-т Красный; стр. 23...,{'source': 'Новосибирск; пр-т Красный; стр. 23...,Новосибирск; пр-т Красный; стр. 236
325009,379198,{'source': 'Новосибирск; пр-т Красный; стр. 23...,{'source': 'Новосибирск; пр-т Красный; стр. 23...,Новосибирск; пр-т Красный; стр. 236


In [67]:
dadata.groupby(['addr'])['id'].count()

addr
Казань; 10-я Союзная ул.; 35             1
Казань; 2-я Газовая ул.; 5к2             1
Казань; 2-я Даурская ул.; 11             1
Казань; 2-я Даурская ул.; 2              2
Казань; 2-я Даурская ул.; 4к3            1
                                        ..
г. Челябинск, ш. Металлургов, д. 80А     1
г. Челябинск, ш. Металлургов, д. 82      1
г. Челябинск, ш. Металлургов, д. 82 А    1
г. Челябинск, ш. Металлургов, д. 84      1
г. Челябинск, ш. Металлургов, д. 84А     1
Name: id, Length: 138311, dtype: int64

In [73]:
addr_group = pd.DataFrame(dadata.groupby(['addr'])['id'].count().sort_values().reset_index())

In [78]:
addr_group.sort_values('id', ascending=False).to_excel('addr_group_sorted.xlsx')

In [79]:
addr_group.sort_values('id', ascending=False)

,addr,id
138310,Новосибирск; р-н Калининский; ул. Объединения;...,1159
138309,Новосибирск; пр-т Красный; стр. 236,1040
138308,Новороссийск; ул. Куникова; лит. 2,684
138307,Новосибирск; ул. Овражная; д. 2А,658
138306,Новосибирск; р-н Кировский; ул. Бородина; д. 2,509
...,...,...
46098,"г. Челябинск, ул. Южная, д. 2А",1
46097,"г. Челябинск, ул. Южная, д. 2Б",1
46096,"г. Челябинск, ул. Южная, д. 2в",1
46095,"г. Челябинск, ул. Южная, д. 3",1


In [81]:
addr_group.iloc[138310,0]

'Новосибирск; р-н Калининский; ул. Объединения; д. 102/4'